Before you begin, execute this cell to import numpy and packages from the D-Wave Ocean suite, and all necessary functions for the gate-model framework you are going to use, whether that is the Forest SDK or Qiskit. In the case of Forest SDK, it also starts the qvm and quilc servers.

In [ ]:
%run -i "assignment_helper.py"
%matplotlib inline

# Loss Functions and Regularization

Let's generate an easily, linearly separable dataset and shuffle it into a training set that we are going to optimize over (2/3 of the data), and a test set where we estimate our generalization performance. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)

c1 = np.random.rand(50, 2)/5
c2 = (-0.6, 0.5) + np.random.rand(50, 2)/5
data = np.concatenate((c1, c2))
labels = np.array([0] * 50 + [1] *50)
plt.figure(figsize=(6, 6))
plt.subplot(111, xticks=[], yticks=[])
plt.scatter(data[:50, 0], data[:50, 1], color='navy')
plt.scatter(data[50:, 0], data[50:, 1], color='c')
idx = np.arange(len(labels))
np.random.shuffle(idx)
# train on a random 2/3 and test on the remaining 1/3
idx_train = idx[:2*len(idx)//3]
idx_test = idx[2*len(idx)//3:]
X_train = data[idx_train]
X_test = data[idx_test]
y_train = labels[idx_train]
y_test = labels[idx_test]

**Exercise 1** (1 point). A multilayer perceptron is a simple neural network that is composed of perceptrons arranged in layers, mapping from some high-dimensional space to a number of classes. In scikit-learn, it minimizes the cross-entropy $L(\theta, S) = -\frac {1}{N}\sum _{i=1}^N y_i \log h(\theta;x_i) + (1-y_i) \log(1-h(\theta;x_i))$, where our training set is $S=\{(x_i, y_i)\}_{i=1}^N$. Train one such network with default parameters. The model is called `MLPClassifier` in scikit-learn. Place the trained model in an object called `model_1`.

In [ ]:
import sklearn
import sklearn.metrics
from sklearn.neural_network import MLPClassifier
metric = sklearn.metrics.accuracy_score
### BEGIN SOLUTION
model_1 = MLPClassifier()
model_1.fit(X_train, y_train)
### END SOLUTION

In [ ]:
assert isinstance(model_1, sklearn.neural_network.MLPClassifier)
assert np.isclose(metric(y_train, model_1.predict(X_train)), 1.0)
assert np.isclose(metric(y_test, model_1.predict(X_test)), 1.0)

**Exercise 2** (2 points). The model can be regularized by an $l_2$ penalty term, which means that we can penalize it for having large weights in the connections between the perceptrons. For the model you trained, the sum of the square norms is the following:

In [ ]:
sum(np.linalg.norm(coefs) for coefs in model_1.coefs_)

The strength of the normalization depends on a hyperparameter `alpha` that you can supply when you instantiate the class. Train a heavily regularized version that will have a lower $l_2$ norm for the weights that maintains the training and test accuracy. Place the trained model in an object called `model_2`.

In [ ]:
### BEGIN SOLUTION
model_2 = MLPClassifier(alpha=10)
model_2.fit(X_train, y_train)
### END SOLUTION

In [ ]:
assert isinstance(model_2, sklearn.neural_network.MLPClassifier)
assert np.isclose(metric(y_train, model_2.predict(X_train)), 1.0)
assert np.isclose(metric(y_test, model_2.predict(X_test)), 1.0)
assert abs(sum(np.linalg.norm(coefs) for coefs in model_1.coefs_)-
           sum(np.linalg.norm(coefs) for coefs in model_2.coefs_)) > 1

In principle, a sparser model (in this case, one with lower $l_2$ norm) provides better guarantees for generalization beyond the training dataset.

# Ensemble methods

Ensembles are all about combining diverse models into a single strong classifier. There are countless ways to construct ensembles. One way to ensure diversity is to train the same family of models, but on different subsets of the training data. This is known as [bagging](https://en.wikipedia.org/wiki/Bootstrap_aggregating) and it is a way of avoiding overfitting. You can combine bagging with other forms of ensembles to construct robust models.

**Exercise 3** (1 point). Random forests do bagging: the algorithm trains many simple decision trees on various subsets of the training data, then averages their output to make predictions. Train a random forest on the above data to maximum accuracy with 20 trees in the ensemble. Place the trained model in an object called `model_3`.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
### BEGIN SOLUTION
model_3 = RandomForestClassifier(n_estimators=20)
model_3.fit(X_train, y_train)
### END SOLUTION

In [ ]:
assert isinstance(model_3, sklearn.ensemble.RandomForestClassifier)
assert model_3.n_estimators == 20
assert np.isclose(metric(y_train, model_3.predict(X_train)), 1.0)
assert np.isclose(metric(y_test, model_3.predict(X_test)), 1.0)

# QBoost

QBoost uses $l_0$ regularization to penalize the number of models in an ensemble and solves this as a discrete optimization problem. We will use the Swiss roll dataset in this section:

In [ ]:
import mpl_toolkits.mplot3d.axes3d as p3
import sklearn.datasets
np.random.seed(0)
data, t = sklearn.datasets.make_swiss_roll(n_samples=500)
labels = np.array(t<t.mean())
idx = np.arange(len(labels))
np.random.shuffle(idx)
# train on a random 2/3 and test on the remaining 1/3
idx_train = idx[:2*len(idx)//3]
idx_test = idx[2*len(idx)//3:]
X_train = data[idx_train]
X_test = data[idx_test]

y_train = 2 * labels[idx_train] - 1  # binary -> spin
y_test = 2 * labels[idx_test] - 1

scaler = sklearn.preprocessing.StandardScaler()
normalizer = sklearn.preprocessing.Normalizer()

X_train = scaler.fit_transform(X_train)
X_train = normalizer.fit_transform(X_train)

X_test = scaler.fit_transform(X_test)
X_test = normalizer.fit_transform(X_test)
fig = plt.figure()
ax = p3.Axes3D(fig)
ax.view_init(7, -80)
for l in np.unique(labels):
    ax.scatter(data[labels == l, 0], data[labels == l, 1], data[labels == l, 2])

**Exercise 4** (1 point). Train a random forest of twenty trees on this dataset until you get accuracy 1 on the training set. You can ignore the accuracy on the test set. Call this `model_4`.

In [ ]:
### BEGIN SOLUTION
model_4 = RandomForestClassifier(n_estimators=20)
model_4.fit(X_train, y_train)
### END SOLUTION

In [ ]:
assert isinstance(model_4, sklearn.ensemble.RandomForestClassifier)
assert model_4.n_estimators == 20
assert np.isclose(metric(y_train, model_4.predict(X_train)), 1.0, rtol=0.1)

Observe the gap between training and test accuracy:

In [ ]:
metric(y_train, model_4.predict(X_train)) - metric(y_test, model_4.predict(X_test))

**Exercise 5** (2 points). The random forest as you trained it gives you twenty weak estimators. Create a function that takes a numpy array of predictions over all weak estimators, the true labels, and a $\lambda$ parameter for regularization, and returns a quadratic binary optimization of the QBoost objective function for dimod. Your return value is a weight matrix as a dictionary. You can assume that the prediction array's shape is the number of models times the number of training points.

In [ ]:
def get_qubo(predictions, y, λ):
    ### BEGIN SOLUTION
    n_models, n_sample = predictions.shape
    w = predictions @ predictions.T/(n_models ** 2)
    wii = n_sample / (n_models ** 2) + λ - 2 * predictions @ y/(n_models)
    w[np.diag_indices_from(w)] = wii
    W = {}
    for i in range(n_models):
        for j in range(i, n_models):
            W[(i, j)] = w[i, j]
    return W
    ### END SOLUTION

In [ ]:
predictions = np.array([[-1, 1], [-1, 1]], dtype=np.float64)
y = [-1, 1]
assert get_qubo(predictions, y, 1) == {(0, 0): -0.5, (0, 1): 0.5, (1, 1): -0.5}
assert get_qubo(predictions, y, 10) == {(0, 0): 8.5, (0, 1): 0.5, (1, 1): 8.5}

Note that the random forest model always predicts 0 or 1 for the individiual weak estimators, so we shift its prediction output to match our spin model.

In [ ]:
predictions = 2*np.array([h.predict(X_train) for h in model_4.estimators_], dtype=np.float64)-1

**Exercise 6** (3 points). Using simulated annealing, find a value for the $\lambda$ parameter that reduces the number of trees used while maintaining the accuracy on the training set. The return value of the sampler should be in an object called `response`. You can use the following function to calculate the prediction of your ensemble:

In [ ]:
def predict(models, weights, X):
    n_data = len(X)
    T = 0
    y = np.zeros(n_data)
    for i, h in enumerate(models):
        y0 = weights[i] * (2*h.predict(X)-1)  # prediction of weak classifier
        y += y0
        T += np.sum(y0)
    y = np.sign(y - T / (n_data*len(models)))
    return y

In [ ]:
### BEGIN SOLUTION
W = get_qubo(predictions, y_train, 15)
sampler = dimod.SimulatedAnnealingSampler()
response = sampler.sample_qubo(W, num_reads=10)
### END SOLUTION

In [ ]:
weights = list(response.first.sample.values())
print('accuracy (train): %5.2f'%(metric(y_train, predict(model_4.estimators_, weights, X_train))))
print('accuracy (test): %5.2f'%(metric(y_test, predict(model_4.estimators_, weights, X_test))))

In [ ]:
assert isinstance(response, dimod.sampleset.SampleSet)
weights = list(response.first.sample.values())
assert sum(weights) < 20
assert np.isclose(metric(y_train, predict(model_4.estimators_, weights, X_train)), 1.0, rtol=0.1)

In principle, you can do the above optimization on a quantum computer and ideally on larger ensembles. This would eventually lead to a smaller gap between the training and test error, that is, it would lower the variance, although it might sacrifice some of the accuracy.